In [1]:
import xarray as xr
import distributed

In [10]:
cluster = distributed.LocalCluster(n_workers=4, threads_per_worker=1, local_directory='/tmp')
client = distributed.Client(cluster)
client

c:\Users\haris\miniconda3\envs\ET\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 49869 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:49869/status,
Dashboard: http://127.0.0.1:49869/status,Workers: 4
Total threads: 4,Total memory: 15.34 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:49870,Workers: 4
Dashboard: http://127.0.0.1:49869/status,Total threads: 4
Started: Just now,Total memory: 15.34 GiB
Comm: tcp://127.0.0.1:49890,Total threads: 1
Dashboard: http://127.0.0.1:49891/status,Memory: 3.84 GiB
Nanny: tcp://127.0.0.1:49873,


# Split ERA5 into training / test

In [11]:
ds_era5 = xr.open_dataset('out.nc',  engine='netcdf4').chunk({'time': 2})

In [12]:
ds_era5

<xarray.Dataset> Size: 3MB
Dimensions:  (time: 248, lat: 21, lon: 21)
Coordinates:
  * time     (time) datetime64[ns] 2kB 2020-12-01 ... 2023-12-31T12:00:00
  * lat      (lat) float64 168B 26.38 26.62 26.88 27.12 ... 30.88 31.12 31.38
  * lon      (lon) float64 168B 75.25 75.5 75.75 76.0 ... 79.5 79.75 80.0 80.25
Data variables:
    z        (time, lat, lon) float64 875kB dask.array<chunksize=(2, 21, 21), meta=np.ndarray>
    t        (time, lat, lon) float64 875kB dask.array<chunksize=(2, 21, 21), meta=np.ndarray>
    t2m      (time, lat, lon) float64 875kB dask.array<chunksize=(2, 21, 21), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-19 16:13:06 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [13]:
ds_era5 = ds_era5.rename({'lat': 'latitude', 'lon': 'longitude'})

In [14]:
len(ds_era5)

3

In [15]:
ds_era5_train = ds_era5.sel(time=slice('2020-12-03', '2022-12-31'))[['t2m']]
ds_era5_test = ds_era5.sel(time=slice('2023-12-03', '2023-12-31'))[['t2m']]

In [16]:
ds_era5_test

<xarray.Dataset> Size: 205kB
Dimensions:    (time: 58, latitude: 21, longitude: 21)
Coordinates:
  * time       (time) datetime64[ns] 464B 2023-12-03 ... 2023-12-31T12:00:00
  * latitude   (latitude) float64 168B 26.38 26.62 26.88 ... 30.88 31.12 31.38
  * longitude  (longitude) float64 168B 75.25 75.5 75.75 ... 79.75 80.0 80.25
Data variables:
    t2m        (time, latitude, longitude) float64 205kB dask.array<chunksize=(2, 21, 21), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-19 16:13:06 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [17]:
ds_era5_train.to_zarr(
    '../data/processed/era5/ds_train',
    encoding={
        't2m': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
    }
)

AttributeError: 'DirectoryStore' object has no attribute 'close'

In [ ]:
t = xr.open_dataset(r"D:\Projects\NIOT\v1\ensemble_transformer\scripts\data\raw\t2m_raw.nc")

In [18]:
ds_era5_test.to_zarr(
    '../data/processed/era5/ds_test',
    encoding={
        't2m': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
    }
)

AttributeError: 'DirectoryStore' object has no attribute 'close'